# Glow-TTS 학습

## 1. 할당된 GPU 확인

In [1]:
import os
GPU_NAME = os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()
os.environ['GPU_NAME'] = GPU_NAME
print(f'GPU: {GPU_NAME}')

GPU: GeForce RTX 2080 Ti
GeForce RTX 2080 Ti
GeForce RTX 2080 Ti
GeForce RTX 2080 Ti


## 2. 필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

이 과정은 약 10분 정도 소요될 수 있습니다.

In [2]:
import sys
from pathlib import Path

In [ ]:
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
%cd TTS
!pip install -q --no-cache-dir -e .

/content
Cloning into 'TTS'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (415/415), done.
remote: Total 447 (delta 56), reused 287 (delta 20), pack-reused 0
Receiving objects: 100% (447/447), 13.77 MiB | 31.33 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/TTS
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 235kB 28.1MB/s 
     |████████████████████████████████| 20.1MB 56.7MB/s 
     |████████████████████████████████| 122kB 63.3MB/s 
     |████████████████████████████████| 1.3MB 22.0MB/s 
     |████████████████████████████████| 3.2MB 55.5MB/s 
     |████████████████████████████████| 71kB 47.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |█████████

## 3. 학습할 데이터셋 불러오기

학습에 사용할 음성 데이터를 본 디렉터리로 가져옵니다.

In [ ]:
%cd TTS
!unzip -q filelists.zip -d ./filelists

/content/TTS


## 4. 사전 학습 데이터 불러오기

In [4]:
%cd TTS
if not Path("./glowtts-v2/config.json").exists():
    !gdown --id 1DMKLdfZ_gzc_z0qDod6_G8fEXj0zCHvC -O glowtts-v2.zip
    !unzip -q glowtts-v2.zip -d ./

[Errno 2] No such file or directory: '/content/TTS'
/home/gpuadmin/yeonsik/TTS/TTS/TTS
Downloading...
From: https://drive.google.com/uc?id=1DMKLdfZ_gzc_z0qDod6_G8fEXj0zCHvC
To: /home/gpuadmin/yeonsik/TTS/TTS/TTS/glowtts-v2.zip
100%|████████████████████████████████████████| 314M/314M [02:14<00:00, 2.33MB/s]


glowtts-v2/config.json 파일 내 경로 수정

In [14]:
# %cd TTS
if not Path("./glowtts-v2/scale_stats_new.npy").exists():
    !python bin/compute_statistics.py "./glowtts-v2/config.json" "./glowtts-v2/scale_stats_new.npy" --data_path "../filelists/wavs/"

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.1
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > There are 38 files.
100%|███████████████████████████████████████████| 38/38 [00:01<00:00, 26.98it/s]
 > Avg mel spec mean: -57.795387268066406
 > Avg mel spec scale: 15.4685640335083
 > Avg linear spec mean: -36.72224426269531
 > Avg lienar spec scale: 15.131887435913086
 > stats saved to ./glowtts-v2/scale_stats_new.npy


## 6. Glow-TTS 학습 진행

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
# !(python TTS/bin/train_glow_tts.py \
#     --continue_path "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/CONTINUE_DIRECTORY")
!(python TTS/bin/train_glow_tts.py \
    --config_path "./TTS/glowtts-v2/config.json" \
    --coqpit.datasets.0.path "./1039samples_1h"  \
    --coqpit.audio.stats_path "./TTS/glowtts-v2/scale_stats_new.npy"  \
    --coqpit.test_sentences_file "./TTS/test_sentences.txt"  \
    --coqpit.output_path "./TTS/glowtts-v2/"  \
    --coqpit.num_loader_workers 2  \
    --coqpit.num_val_loader_workers 2  \
    --restore_path "./TTS/glowtts-v2/model_file.pth.tar")

 > Using CUDA:  True
 > Number of GPUs:  1
 > Git Hash: 3aa165a
 > Experiment folder: ./TTS/glowtts-v2/glowtts-v2-December-17-2021_07+57PM-3aa165a
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.1
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:./TTS/glowtts-v2/scale_stats_new.npy
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 481 files in /home/gpuadmin/yeonsik/TTS/TTS/500samples_30m
 > Using model: glow_tts
/home/gpuadmin/yeonsik/TTS/TTS/TTS/tts/layers/glow_tts/glow.py:84: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be r